In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [3]:
# Leitura dos arquivos CSV 
sample_submission = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/sample_submission.csv')
test = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/test.csv')
train = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/train.csv')


In [4]:
train.head()


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
3,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0
4,ISIC_0074311,IP_2950485,female,40.0,lower extremity,unknown,benign,0


In [5]:
train.tail()


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
28979,ISIC_9999134,IP_6526534,male,50.0,torso,unknown,benign,0
28980,ISIC_9999320,IP_3650745,male,65.0,torso,unknown,benign,0
28981,ISIC_9999515,IP_2026598,male,20.0,lower extremity,unknown,benign,0
28982,ISIC_9999666,IP_7702038,male,50.0,lower extremity,unknown,benign,0
28983,ISIC_9999806,IP_0046310,male,45.0,torso,nevus,benign,0


In [6]:
train.describe(include='all')


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
count,28984,28984,28928,28925.000000,28518,28984,28984,28984.000000
unique,28984,2056,2,NaN,6,9,2,NaN
top,ISIC_2637011,IP_4382720,male,NaN,torso,unknown,benign,NaN
freq,1,104,14953,NaN,14781,23722,28471,NaN
mean,NaN,NaN,NaN,48.886776,NaN,NaN,NaN,0.017699
std,NaN,NaN,NaN,14.408609,NaN,NaN,NaN,0.131859
min,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,40.000000,NaN,NaN,NaN,0.000000
50%,NaN,NaN,NaN,50.000000,NaN,NaN,NaN,0.000000
75%,NaN,NaN,NaN,60.000000,NaN,NaN,NaN,0.000000


In [7]:
test.describe(include='all')


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
count,4142,4142,4133,4133.000000,4081
unique,4142,1457,2,NaN,6
top,ISIC_0052212,IP_4419570,male,NaN,torso
freq,1,22,2127,NaN,2064
mean,NaN,NaN,NaN,48.752722,NaN
std,NaN,NaN,NaN,14.182254,NaN
min,NaN,NaN,NaN,15.000000,NaN
25%,NaN,NaN,NaN,40.000000,NaN
50%,NaN,NaN,NaN,50.000000,NaN
75%,NaN,NaN,NaN,60.000000,NaN


In [8]:
cat_cols = ['patient_id', 'sex', 'anatom_site_general_challenge', 'diagnosis', 'benign_malignant']
print('Contagens dos atributos categóricos do conjunto de treino')
for col in cat_cols:
    print('Contagem de valores da coluna {}'.format(col))
    print(train[col].value_counts().head(20))
    print('='*80)


Contagens dos atributos categóricos do conjunto de treino
Contagem de valores da coluna patient_id
patient_id
IP_4382720    104
IP_0656529    102
IP_4938382    101
IP_7279968     98
IP_4479736     97
IP_9147454     94
IP_3057277     93
IP_1139701     92
IP_4488328     92
IP_7330399     89
IP_5889408     88
IP_6245507     86
IP_1969685     85
IP_2153088     83
IP_4419570     80
IP_0660290     79
IP_6342052     79
IP_5805281     79
IP_6071452     79
IP_0093378     75
Name: count, dtype: int64
Contagem de valores da coluna sex
sex
male      14953
female    13975
Name: count, dtype: int64
Contagem de valores da coluna anatom_site_general_challenge
anatom_site_general_challenge
torso              14781
lower extremity     7334
upper extremity     4356
head/neck           1622
palms/soles          316
oral/genital         109
Name: count, dtype: int64
Contagem de valores da coluna diagnosis
diagnosis
unknown                               23722
nevus                                  4559
mela

In [9]:
cat_cols = ['patient_id', 'sex', 'anatom_site_general_challenge']
print('Contagens dos atributos categóricos do conjunto de teste')
for col in cat_cols:
    print('Contagem de valores da coluna {}'.format(col))
    print(test[col].value_counts().head(20))
    print('='*80)


Contagens dos atributos categóricos do conjunto de teste
Contagem de valores da coluna patient_id
patient_id
IP_4419570    22
IP_4479736    18
IP_7279968    17
IP_6245507    16
IP_1969685    15
IP_4938382    14
IP_9896679    13
IP_0660290    13
IP_6512345    13
IP_3713422    12
IP_7330399    12
IP_4488328    12
IP_0656529    12
IP_0621614    12
IP_5805281    12
IP_7842431    11
IP_7890415    11
IP_1197949    11
IP_9558959    11
IP_4382720    11
Name: count, dtype: int64
Contagem de valores da coluna sex
sex
male      2127
female    2006
Name: count, dtype: int64
Contagem de valores da coluna anatom_site_general_challenge
anatom_site_general_challenge
torso              2064
lower extremity    1083
upper extremity     627
head/neck           233
palms/soles          59
oral/genital         15
Name: count, dtype: int64


In [10]:
submission = sample_submission


In [11]:
submission.to_csv('submission.csv', index=False)


In [12]:
pd.read_csv('submission.csv')


,image_name,target
0,ISIC_0052212,0
1,ISIC_0076545,0
2,ISIC_0085172,0
3,ISIC_0086709,0
4,ISIC_0109568,0
...,...,...
4137,ISIC_9987485,0
4138,ISIC_9989936,0
4139,ISIC_9994122,0
4140,ISIC_9994390,0
